In [1]:
import os
import time
import itertools
import numpy as np
import pandas as pd
import pickle
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import catboost
from catboost import CatBoostClassifier, CatBoostRegressor
import optuna
from sklearn.model_selection import train_test_split, KFold

from Dataset_Construction import Balance_Ratio 
from Sampling import label_divide
from AdaClassifier import train_set, multiple_set, print_badC, bad_plot, line_chart, cf_matrix
from AdaRegressor import AUC, PR_curve, multiple_curve, PR_matrix, best_threshold 
from Aging_Score import score1
from XGBoost import optuna_history

os.chdir('C:/Users/user/Desktop/Darui_R08621110') 
os.getcwd()

'C:\\Users\\user\\Desktop\\Darui_R08621110'

### Boosting model

In [2]:
def CatBoostC(train_x, test_x, train_y, test_y, config, cat_feature):
    
    clf = CatBoostClassifier(**config, verbose = 0)
    clf.fit(train_x, train_y, cat_features = cat_feature)
    predict_y = clf.predict(test_x)
    result = pd.DataFrame({'truth': test_y, 'predict': predict_y})
    
    return result


def CatBoostR(train_x, test_x, train_y, test_y, config, cat_feature):
    
    reg = CatBoostRegressor(**config, verbose = 0)
    reg.fit(train_x, train_y, cat_features = cat_feature)
    predict_y = reg.predict(test_x)
    result = pd.DataFrame({'truth': test_y, 'predict': predict_y})

    return result

### Run all dataset

In [3]:
def runall_CatBoostC(num_set, trainset_x, test_x, trainset_y, test_y, config, cat_feature, record_bad = True):
    
    table_set = pd.DataFrame()
    bad_set = pd.DataFrame()
    judge = list(config.keys())[0]

    for i in range(num_set):
        print('\n', f'Dataset {i}:')
        
        if isinstance(config[judge], dict) :
            best_config = config[f'set{i}']
        else :
            best_config = config
        
        result = CatBoostC(trainset_x[f'set{i}'], test_x, trainset_y[f'set{i}'], test_y, best_config, cat_feature)
        table = cf_matrix(result, trainset_y[f'set{i}'])
        table_set = pd.concat([table_set, table]).rename(index = {0: f'dataset {i}'})
        
        if record_bad:
            bad_table = print_badC(result, test_x, Bad_Types) 
            bad_set = pd.concat([bad_set, bad_table]).rename(index = {0: f'dataset {i}'})

    if record_bad:
        return table_set, bad_set
    else:
        return table_set
    
    
def runall_CatBoostR(num_set, trainset_x, test_x, trainset_y, test_y, config, cat_feature, thres_target = 'Recall', 
                     threshold = 0.8, record_bad = True):
    
    table_set = pd.DataFrame()
    bad_set = pd.DataFrame()
    pr_dict = {}
    judge = list(config.keys())[0]

    for i in range(num_set):
        print('\n', f'Dataset {i}:')
        
        if isinstance(config[judge], dict) :
            best_config = config[f'set{i}']
        else :
            best_config = config

        predict = CatBoostR(trainset_x[f'set{i}'], test_x, trainset_y[f'set{i}'], test_y, best_config, cat_feature)
        pr_matrix = PR_matrix(predict, trainset_y[f'set{i}'])
        pr_dict[f'set{i}'] = pr_matrix
        
        best_data, best_thres = best_threshold(pr_matrix, target = thres_target, threshold = threshold)
        table_set = pd.concat([table_set, best_data]).rename(index = {best_data.index.values[0]: f'dataset {i}'})
        
        if record_bad:
            bad_table = print_badC(predict, test_x, Bad_Types, threshold = best_thres)
            bad_set = pd.concat([bad_set, bad_table]).rename(index = {0: f'dataset {i}'})
    
    if record_bad:
        return pr_dict, table_set, bad_set
    else:
        return pr_dict, table_set

## Data Processing

In [4]:
###bad types###
bad = pd.read_csv('event/Bad_Types.csv').iloc[:, 1:]
Bad_Types = {bad.cb[i]:i for i in range (len(bad))}
print('Total bad types:', len(bad))

###single dataset###
test = pd.read_csv('event/TestingSet_0.csv').iloc[:, 2:]
train = pd.read_csv('event/TrainingSet_new.csv').iloc[:, 2:]
print('\ntraining data:', train.shape, '\nBalance Ratio:', Balance_Ratio(train))
print('\ntesting data:', test.shape, '\nBalance Ratio:', Balance_Ratio(test))

train_x, train_y, test_x, test_y = label_divide(train, test, 'GB')

###multiple dataset###
data_dict = multiple_set(num_set = 10)
trainset_x, trainset_y = train_set(data_dict, num_set = 10, label = 'GB')
test_x, test_y = label_divide(test, None, 'GB', train_only = True)


#####for runhist dataset#####
# bad = pd.read_csv('run_bad_types.csv').iloc[:, 1:]
# Bad_Types = {bad.cb[i]:i for i in range (len(bad))}
# print('Total bad types:', len(bad))

run_test = pd.read_csv('test_runhist.csv').iloc[:, 2:]
run_test_x, run_test_y = label_divide(run_test, None, 'GB', train_only = True)
print('\n', 'Dimension of run test:', run_test.shape)

Total bad types: 62

training data: (77138, 83) 
Balance Ratio: 18.17902

testing data: (55903, 83) 
Balance Ratio: 3104.72222
Dimension of dataset 0 : (157140, 234)  balance ratio: 630.08434
Dimension of dataset 1 : (4904, 234)  balance ratio: 1.0
Dimension of dataset 2 : (4648, 234)  balance ratio: 1.0
Dimension of dataset 3 : (5196, 234)  balance ratio: 1.0
Dimension of dataset 4 : (4706, 234)  balance ratio: 1.0
Dimension of dataset 5 : (4985, 234)  balance ratio: 0.998
Dimension of dataset 6 : (4816, 234)  balance ratio: 1.07051
Dimension of dataset 7 : (4980, 234)  balance ratio: 1.0
Dimension of dataset 8 : (4980, 234)  balance ratio: 1.0
Dimension of dataset 9 : (1079, 234)  balance ratio: 3.33333

 10 datasets are loaded.

Labels of  10 datasets are divided.

 Dimension of run test: (48650, 234)


### Classifier

In [ ]:
#table_set1, bad_set1 = runall_CatBoostC(9, trainset_x, test_x, trainset_y, test_y, cat_feature = [], event_clf_param)
table_set1 = runall_CatBoostC(10, trainset_x, run_test_x, trainset_y, run_test_y, best_paramC, cat_feature = [],
                              record_bad = False)
line_chart(table_set1, title = 'CatBoost Classifier')
#bad_plot(bad_set1)

In [ ]:
table_set1

### Regressor

In [ ]:
# pr_dict, table_set2, bad_set2 = runall_CatBoostR(9, trainset_x, test_x, trainset_y, test_y, event_reg_param, 
#                                                  thres_target = 'Recall', threshold = 0.8)
pr_dict, table_set2 = runall_CatBoostR(10, trainset_x, run_test_x, trainset_y, run_test_y, best_paramR, cat_feature = [], 
                                       thres_target = 'Recall', threshold = 0.8, record_bad = False)
line_chart(table_set2, title = 'CatBoost Regressor')
#bad_plot(bad_set2)

In [ ]:
multiple_curve(3, 3, pr_dict, table_set2, target = 'Aging Rate')
multiple_curve(3, 3, pr_dict, table_set2, target = 'Precision')
table_set2

## Optimization

### Optuna

In [5]:
def objective_creator(train_data, mode, cat_feature, num_valid = 3, label = 'GB') :

    def objective(trial) :
    
        param_1 = {
            #'objective': 'CrossEntropy',
            'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
            'iterations': trial.suggest_int('iterations', 100, 300, step = 50),
            'depth': trial.suggest_int('depth', 2, 12, step = 2),
            'learning_rate': trial.suggest_float('learning_rate', 0.025, 0.325, step = 0.05),
            'subsample': trial.suggest_float('subsample', 0.5, 0.9, step = 0.2),
            'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-4, 10) # lambda
        }

        if param_1['grow_policy'] == 'Depthwise' :
            param_2 = {
                'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 3, 30, step = 3)
            }
            param = {**param_1, **param_2}
        
        elif param_1['grow_policy'] == 'Lossguide' :
            param_3 = {
                'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 3, 30, step = 3),
                'max_leaves': trial.suggest_int('max_leaves', 15, 50, step = 5)
            }
            param = {**param_1, **param_3}
        
        else :
            param = param_1

        result_list = []
        for i in range(num_valid):

            train_x, train_y = label_divide(train_data, None, label, train_only = True)
            train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size = 0.25)

            if mode == 'C':
                result = CatBoostC(train_x, valid_x, train_y, valid_y, param, cat_feature)
                table = cf_matrix(result, valid_y)
                recall = table['Recall']
                aging = table['Aging Rate']
                effi = table['Efficiency']
            
                #result_list.append(effi)
                result_list.append(recall - 0.1*aging)

            elif mode == 'R':
                result = CatBoostR(train_x, valid_x, train_y, valid_y, param, cat_feature)
                pr_matrix = PR_matrix(result, valid_y)

                #best_data, _ = best_threshold(pr_matrix, target = 'Recall', threshold = 0.8)
                #aging = best_data['Aging Rate']
                #result_list.append((-1)*aging)

                auc = AUC(pr_matrix['Recall'], pr_matrix['Aging Rate'])
                result_list.append((-1)*auc)

        return np.mean(result_list)

    return objective


def all_optuna(num_set, all_data, mode, cat_feature, TPE_multi, n_iter, num_valid = 3, return_addition = True) :

    best_param = {}
    #all_study = {}
    all_score = {}
    for i in tqdm(range(num_set)) :
        
        ##### define objective function and change optimized target dataset in each loop #####
        objective = objective_creator(train_data = data_dict[f'set{i}'], mode = mode, cat_feature = cat_feature,
                                      num_valid = num_valid)
        
        ##### optimize one dataset in each loop #####
        print(f'Dataset{i} :')
        
        study = optuna.create_study(sampler = optuna.samplers.TPESampler(multivariate = TPE_multi), 
                                       direction = 'maximize')
        study.optimize(objective, n_trials = n_iter, show_progress_bar = True, gc_after_trial = True)
        #n_trials or timeout
        best_param[f'set{i}'] = study.best_trial.params
        
        ##### return score and entire params for score plot or feature importance #####
        if return_addition :
            collect_score = []
            [collect_score.append(x.values) for x in study.trials]
            #all_study[f'set{i}'] = study
            all_score[f'set{i}'] = collect_score 
        
        print(f"Sampler is {study.sampler.__class__.__name__}")
    
    ##### store the best hyperparameters #####
    multi_mode = 'multivariate' if TPE_multi else 'univariate'
    with open(f'runhist_array_m2m5_CatBoost{mode}_{multi_mode}-TPE_{n_iter}.data', 'wb') as f:
        pickle.dump(best_param, f)
    
    if return_addition :
        return best_param, all_score, #all_study
    else :
        return best_param

In [ ]:
best_paramC, all_scoreC = all_optuna(num_set = 10, all_data = data_dict, mode = 'C', cat_feature = [],
                                     TPE_multi = False, n_iter = 250)
best_paramR, all_scoreR = all_optuna(num_set = 10, all_data = data_dict, mode = 'R', cat_feature = [],
                                     TPE_multi = False, n_iter = 250)

  0%|          | 0/10 [00:00<?, ?it/s]

[I 2021-08-14 23:47:17,209] A new study created in memory with name: no-name-dabf0645-fff3-4336-a64d-37b02ba4faef


Dataset0 :


C:\Users\user\anaconda3\envs\aging\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/250 [00:00<?, ?it/s]

...

[I 2021-08-16 11:12:01,773] Trial 127 finished with value: -0.11726599068958156 and parameters: {'grow_policy': 'Depthwise', 'iterations': 150, 'depth': 8, 'learning_rate': 0.025, 'subsample': 0.7, 'l2_leaf_reg': 4.527700670548754, 'min_data_in_leaf': 15}. Best is trial 22 with value: -0.10363404535857527.
[I 2021-08-16 11:12:04,129] Trial 128 finished with value: -0.1281246739697444 and parameters: {'grow_policy': 'Depthwise', 'iterations': 300, 'depth': 10, 'learning_rate': 0.07500000000000001, 'subsample': 0.5, 'l2_leaf_reg': 7.057628587947929, 'min_data_in_leaf': 21}. Best is trial 22 with value: -0.10363404535857527.
[I 2021-08-16 11:12:08,039] Trial 129 finished with value: -0.10610967735967736 and parameters: {'grow_policy': 'Lossguide', 'iterations': 300, 'depth': 12, 'learning_rate': 0.125, 'subsample': 0.7, 'l2_leaf_reg': 0.2400423821404822, 'min_data_in_leaf': 27, 'max_leaves': 45}. Best is trial 22 with value: -0.10363404535857527.
[I 2021-08-16 11:13:03,554] Trial 130 fini

[I 2021-08-16 11:14:14,491] Trial 152 finished with value: -0.11886963766021946 and parameters: {'grow_policy': 'Depthwise', 'iterations': 300, 'depth': 12, 'learning_rate': 0.125, 'subsample': 0.7, 'l2_leaf_reg': 0.02079185721821887, 'min_data_in_leaf': 27}. Best is trial 22 with value: -0.10363404535857527.
[I 2021-08-16 11:14:17,507] Trial 153 finished with value: -0.12358608424302932 and parameters: {'grow_policy': 'Depthwise', 'iterations': 300, 'depth': 12, 'learning_rate': 0.125, 'subsample': 0.7, 'l2_leaf_reg': 0.03731514031283117, 'min_data_in_leaf': 27}. Best is trial 22 with value: -0.10363404535857527.
[I 2021-08-16 11:14:21,154] Trial 154 finished with value: -0.12239121635296502 and parameters: {'grow_policy': 'Depthwise', 'iterations': 300, 'depth': 12, 'learning_rate': 0.125, 'subsample': 0.7, 'l2_leaf_reg': 0.01937535343995845, 'min_data_in_leaf': 15}. Best is trial 22 with value: -0.10363404535857527.
[I 2021-08-16 11:14:24,523] Trial 155 finished with value: -0.13972

[I 2021-08-16 11:15:27,164] Trial 177 finished with value: -0.11977112151504461 and parameters: {'grow_policy': 'Depthwise', 'iterations': 300, 'depth': 12, 'learning_rate': 0.17500000000000002, 'subsample': 0.7, 'l2_leaf_reg': 5.906901441219199, 'min_data_in_leaf': 24}. Best is trial 175 with value: -0.10022968257406202.
[I 2021-08-16 11:15:29,977] Trial 178 finished with value: -0.11406588987884643 and parameters: {'grow_policy': 'Depthwise', 'iterations': 300, 'depth': 12, 'learning_rate': 0.17500000000000002, 'subsample': 0.5, 'l2_leaf_reg': 7.521412245596064, 'min_data_in_leaf': 24}. Best is trial 175 with value: -0.10022968257406202.
[I 2021-08-16 11:15:33,976] Trial 179 finished with value: -0.12433400816421163 and parameters: {'grow_policy': 'Lossguide', 'iterations': 300, 'depth': 12, 'learning_rate': 0.17500000000000002, 'subsample': 0.7, 'l2_leaf_reg': 2.4274918133502044, 'min_data_in_leaf': 24, 'max_leaves': 45}. Best is trial 175 with value: -0.10022968257406202.
[I 2021-0

[I 2021-08-16 11:16:34,976] Trial 202 finished with value: -0.12247144894714339 and parameters: {'grow_policy': 'Lossguide', 'iterations': 300, 'depth': 8, 'learning_rate': 0.125, 'subsample': 0.7, 'l2_leaf_reg': 6.358341345945336, 'min_data_in_leaf': 27, 'max_leaves': 45}. Best is trial 175 with value: -0.10022968257406202.
[I 2021-08-16 11:16:37,954] Trial 203 finished with value: -0.12100973310871589 and parameters: {'grow_policy': 'Depthwise', 'iterations': 300, 'depth': 12, 'learning_rate': 0.125, 'subsample': 0.7, 'l2_leaf_reg': 0.141039492108264, 'min_data_in_leaf': 27}. Best is trial 175 with value: -0.10022968257406202.
[I 2021-08-16 11:16:39,304] Trial 204 finished with value: -0.12787852065629843 and parameters: {'grow_policy': 'Lossguide', 'iterations': 100, 'depth': 12, 'learning_rate': 0.225, 'subsample': 0.7, 'l2_leaf_reg': 1.3652649558243906, 'min_data_in_leaf': 24, 'max_leaves': 30}. Best is trial 175 with value: -0.10022968257406202.
[I 2021-08-16 11:16:41,867] Trial 

[I 2021-08-16 11:18:35,041] Trial 227 finished with value: -0.12025856347015662 and parameters: {'grow_policy': 'Depthwise', 'iterations': 300, 'depth': 10, 'learning_rate': 0.125, 'subsample': 0.7, 'l2_leaf_reg': 0.00011400295116972836, 'min_data_in_leaf': 30}. Best is trial 175 with value: -0.10022968257406202.
[I 2021-08-16 11:18:39,191] Trial 228 finished with value: -0.11617204976847917 and parameters: {'grow_policy': 'Lossguide', 'iterations': 300, 'depth': 12, 'learning_rate': 0.125, 'subsample': 0.7, 'l2_leaf_reg': 0.0038946640827090075, 'min_data_in_leaf': 24, 'max_leaves': 50}. Best is trial 175 with value: -0.10022968257406202.
[I 2021-08-16 11:18:42,367] Trial 229 finished with value: -0.1255142984219852 and parameters: {'grow_policy': 'Depthwise', 'iterations': 300, 'depth': 12, 'learning_rate': 0.07500000000000001, 'subsample': 0.7, 'l2_leaf_reg': 0.16756431223990154, 'min_data_in_leaf': 27}. Best is trial 175 with value: -0.10022968257406202.
[I 2021-08-16 11:18:44,976] 

In [ ]:
##### optimization history plot #####
optuna_history(best_paramC, all_scoreC, model = 'CatBoost Classifier')
            
##### best hyperparameter table #####
param_table = pd.DataFrame(best_paramC).T
param_table